In [2]:
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [3]:
tf.__version__

'1.13.1'

In [4]:
keras.__version__

'2.2.4-tf'

In [5]:
model = keras.Sequential()
# 第一层是输入层，无需制定输入的shape，会自动匹配，也不需要activation？
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation="softmax"))

In [6]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [7]:
data = np.random.random((1000,32))
label = np.random.random((1000,10))
val_data = np.random.random((100, 32))
val_label = np.random.random((100, 10))

# 使用data/label训练模型

In [ ]:
history = model.fit(data, label, epochs=10, batch_size=100)

In [8]:
model.fit(data, label, epochs=10, batch_size=32,
          validation_data=(val_data, val_label))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 1000 samples, validate on 100 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 1s 1ms/sample - loss: 11.5669 - acc: 0.1120 - val_loss: 11.5702 - val_acc: 0.0800
Epoch 2/10
1000/1000 [==============================] - 0s 391us/sample - loss: 11.5326 - acc: 0.1160 - val_loss: 11.5676 - val_acc: 0.0500
Epoch 3/10
1000/1000 [==============================] - 0s 425us/sample - loss: 11.5219 - acc: 0.1260 - val_loss: 11.5693 - val_acc: 0.1000
Epoch 4/10
1000/1000 [==============================] - 0s 367us/sample - loss: 11.5141 - acc: 0.1520 - val_loss: 11.5681 - val_acc: 0.0600
Epoch 5/10
1000/1000 [==============================] - 0s 372us/sample - loss: 11.5100 - acc: 0.1440 - val_loss: 11.5736 - val_acc: 0.1100
Epoch 6/10
1000/1000

# 使用Dataset训练模型
直接使用data/label训练模型的时候，传入fit方法的data/label是完整数据集。使用Dataset训练模型时，传入fit的数据集必须经过batch/repeat处理。在使用Dataset训练模型时，所传入的steps_per_epoch参数意为：在每轮（epoch）训练中，每次喂入多少个批次的数据。因此，假设训练数据有1000行，每个批次100行，那么当steps_per_epoch=10时，一轮训练完毕正好使用完所有数据。

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((data,label))
print(dataset.output_shapes)
dataset = dataset.batch(100)
dataset = dataset.repeat()

#history = model.fit(dataset, epochs=10, steps_per_epoch=40)

(TensorShape([Dimension(32)]), TensorShape([Dimension(10)]))


In [13]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset.batch(32).repeat()

model.fit(dataset,epochs=10,steps_per_epoch=20,
         validation_data=val_dataset,
         validation_steps=3)

Epoch 1/10
16/20 [=======================>......] - ETA: 0s - loss: 11.4445 - acc: 0.1969

CancelledError:  [Op:StatefulPartitionedCall]